# The London Railway Network

The cell below defines the abstract class whose API you will need to impement. Do NOT modify it.

In [13]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    # returns the minimum number of stops to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # numStops : int
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    # returns the minimum distance in miles to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # minDistance : float
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    # given an unordered list of station names, returns a new railway line 
    # (represented as a list of adjacent station names), connecting all such stations 
    # and such that the sum of the distances (in miles) between adjacent stations is minimised
    # inputList : set<str>
    # outputList : list<str>
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [14]:
# Queue

'''
To implement a 
'''

class QueueNode:
    def __init__(self, data = None):
        self.data = data
        self.next = None
        self.prev = None

class MyQueue:

    def __init__(self):
        self.size = 0
        self.head = None
        self.tail = None

    def enqueue(self, data):
        node = QueueNode(data)
        if (self.head == None):
            self.head = node
            self.tail = node
        else:
            self.tail.next = node
            node.prev = self.tail
            self.tail = node
        self.size += 1

    def dequeue(self):
        data = self.head.data
        self.head = self.head.next
        self.size -= 1
        if (self.size == 0):
            self.tail = self.head
        return data

    def isEmpty(self):
        return self.size == 0

    def size(self):
        return self.size



In [15]:
# Min-Oriented Priority Queue using Binary Heap

class MinPQ:

    def __init__(self):
        self.size = 0
        self.list = [None]

    def enqueue(self, value, key):
        self.size += 1
        # self.array[self.size] = (value, key)
        self.list.append((value, key))
        self.swim(self.size)

    def swim(self, i):
        while (i > 1 and self.list[i//2][1] > self.list[i][1]):
            self.swap(i, i//2)
            i = i // 2

    def dequeue(self):
        min = self.list[1]
        self.swap(1, self.size)
        self.size -= 1
        self.list.pop()
        self.sink(1)
        return min

    def sink(self, i):
        while (i*2 <= self.size):
            childPos = i*2
            if (childPos < self.size and self.list[childPos][1] > self.list[childPos + 1][1]):
                childPos += 1
            if (self.list[i][1] < self.list[childPos][1]): 
                break
            self.swap(i, childPos)
            i = childPos

    def swap(self, i, j):
        self.list[i], self.list[j] = self.list[j], self.list[i]

    def isEmpty(self):
        return self.size == 0




        


In [16]:
import csv
from math import radians, sin, cos, asin, sqrt

# ADD YOUR DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE



class DataLoader:

    '''
    In my DataLoader class I decied to store a pèython dictionary stationsDict containing all the information about stations
    using the following format: {"stationName" : [ keyForGraph, lat, lon ], ...}. The reason behind chosing a dictionary is
    that it allows us to retrieve data in O(1) on average. This will be extremely useful in every situation in which we need
    to access by name information about a station. This occures, for example, when reading the "londonrailwaylines.csv" file
    to load the edges. The worstcase scenario for a dictionary retrival is O(N), where N is the number of stations. However,
    this case is extremely rare, because of the way the hasmap implementing the python default dictionary is structured.  
    '''

    def __init__(self):
        self.stationsDict = {} # Dictionary of tipe {"stationName" : [ keyForGraph, lat, lon ]}
        self.stationsCount = -1
        self.graph = None

    def distance(self, longitude1, latitude1, longitude2, latitude2):
        
        longitude1 = radians(longitude1)
        longitude2 = radians(longitude2)
        latitude1 = radians(latitude1)
        latitude2 = radians(latitude2)

        KmToMiles = 0.621371
        
        # Haversine formula 
        dlon = longitude2 - longitude1 
        dlat = latitude2 - latitude1
        x = sin(dlat / 2)**2 + cos(latitude1) * cos(latitude2) * sin(dlon / 2)**2
        c = 2 * asin(sqrt(x)) 
        radius = 6371
        return(c * radius * KmToMiles)

    def loadFiles(self):

        with open('londonstations.csv') as stationsFile:
            reader = csv.reader(stationsFile, delimiter=',')
            for row in reader:
                if (self.stationsCount == -1):
                    self.stationsCount += 1
                    continue
                self.stationsDict[row[0]] = [self.stationsCount, float(row[1]), float(row[2])]
                self.stationsCount += 1

        self.graph = WeightedGraph(self.stationsCount)

        with open('londonrailwaylines.csv') as linesFile:
            reader = csv.reader(linesFile, delimiter=',')
            lineCount = 0
            for row in reader:
                if (lineCount == 0): 
                    lineCount += 1
                    continue
                lineName = row[0]
                fromStation = row[1]
                toStation = row[2]
                fromStationValue = self.stationsDict.get(fromStation)
                toStationValue = self.stationsDict.get(toStation)

                dist = self.distance(fromStationValue[2], fromStationValue[1], toStationValue[2], toStationValue[1])
                edge = Edge(fromStationValue[0], toStationValue[0], dist, lineName)
                self.graph.addEdge(edge)
                lineCount += 1

        return self.graph, self.stationsDict



class WeightedGraph:

    def __init__(self, V):
        self.V = V
        self.adj = []
        for _ in range (0, V):
            self.adj.append([])

    def addEdge(self, e):
        v = e.endPoint()
        w = e.otherEndPoint(v)
        self.adj[v].append(e)
        self.adj[w].append(e)

    def adjList(self, v):
        return self.adj[v]

    def getVertexCount(self):
        return self.V

    def addDirectedEdge(self, e, v):
        w = e.otherEndPoint(v)
        self.adj[v].append(e)

class Edge:

    def __init__(self, v, w, weight, lineName):
        self.v = v
        self.w = w
        self.weight = weight
        self.lineName = lineName

    def getLineName(self):
        return self.lineName

    def endPoint(self):
        return self.v

    def otherEndPoint(self, vertex):
        if vertex == self.v: return self.w
        else: return self.v

    def compareTo(self, edge):
        if (self.weight < edge.weight): return -1
        elif (self.weight < edge.weight): return +1
        else: return 0

    def getWeight(self):
        return self.weight

class MinStopsBFS:

    def __init__(self, graph):
        self.graph = graph
        self.distToSource = [-1] * self.graph.getVertexCount()
    
    def minStops(self, start, end):
        queue = MyQueue()    # Implement a queue
        queue.enqueue(start)

        self.distToSource[start] = 0

        while (not queue.isEmpty()):
            v = queue.dequeue()
            for edge in self.graph.adjList(v):
                w = edge.otherEndPoint(v)
                if (self.distToSource[w] == -1):
                    queue.enqueue(w)
                    self.distToSource[w] = self.distToSource[v] + 1
        return self.distToSource[end]

class DijkstraSP:

    def __init__(self, graph, source):
        self.distTo = [float('inf') for v in range(0, graph.getVertexCount())]
        self.distTo[source] = 0
        self.visited = [False for v in range(0, graph.getVertexCount())]
        self.graph = graph

        self.pq = MinPQ()
        self.pq.enqueue(source, 0)


        while (not self.pq.isEmpty()):
            curVertex = self.pq.dequeue()[0]
            if (self.visited[curVertex]): continue
            for edge in self.graph.adjList(curVertex):
                self.relax(edge, curVertex)

            self.visited[curVertex] = True


    def relax(self, edge, fromVertex):
        toVertex = edge.otherEndPoint(fromVertex)
        if (self.distTo[toVertex] > self.distTo[fromVertex] + edge.getWeight()):
            self.distTo[toVertex] = self.distTo[fromVertex] + edge.getWeight()
            self.pq.enqueue(toVertex, self.distTo[toVertex])

    def minDistances(self):
        return self.distTo

class ShortestPath:

    def __init__(self, graph, source):
        self.dijkstra = DijkstraSP(graph, source)

    def findMinDistance(self, destination):
        distTo = self.dijkstra.minDistances()
        return distTo[destination]

class PrimMST:
    def __init__(graph):
        self.marked = [False for vertex in range (0, graph.getVertexCount())]
        self.mst = MyQueue()
        self.pq = MinPQ()
        self.visit(graph, 0)

        while(not self.pq.isEmpty() and self.mst.size() < (graph.getVertexCount() - 1)):
            edge = self.pq.dequeue()
            v = edge.endPoint()
            w = edge.otherEndPoint(v)
            if (self.marked[v] and self.marked[w]): continue
            self.mst.enqueue(edge)
            if (not self.marked[v]): visit(graph, v)
            if (not self.marked[w]): visit(graph, w)

    def visit(self, graph, v):
        self.marked[v] = True
        for edge in graph.adjList(v):
            if (not self.marked[e.otherEndPoint(v)]):
                self.pq.enqueue(edge, edge.getWeight())

    def edges(self):
        return self.mst


class NewRailwayConstructorTSP:

    '''
    Since to compute the Salesman Travelling Algorithm I will need the distance between two stations several times, I decided to compute all the distances ones and access them via a matrix. This will avoid calculating tyhe distance N! times, where N is the number of stations in the input, but only N^2 times. This will use some space (the matrix) but it will be relatively small as it only requires N^2 spaces. 
    '''

    def __init__(self, stations, inputList):
        # Construct a weighted graph in matrix format
        self.inputList = inputList
        self.stations = stations
        self.graphMatrix = [[] for i in range(len(self.inputList))]
        loader = DataLoader()
        for i in range(0, len(self.inputList)):
            fromStationValue = self.stations.get(self.inputList[i])
            for j in range(0, len(self.inputList)):
                toStationValue = self.stations.get(self.inputList[j])
                dist = loader.distance(fromStationValue[2], fromStationValue[1], toStationValue[2], toStationValue[1])
                self.graphMatrix[i].append(dist)

    '''
    To find the optimal order of stations to build a new line minimizing the distance I used a modified version of the Salesman Travelling Algorithm. The problem is NP-hard  and has a cost of O(N!), where N is the number of stations in the input, therefore can only be used with at most roughly 20 stations. My algorithm modifies the Salesman Travelling Algorithm by not precluding a single point as beginning and end. This allows the new railway line to start in any of the stations in the input and does not create a cycle (as we are required to create a straight and non-branching line).
    '''

    def findNewLine(self):
        generator = self.permutations([i for i in range(len(self.inputList))])

        minDist = float('inf')
        minPath = None
        result = []
        for permutation in generator:
            currentDist = 0
            for i in range(len(permutation) - 1):
                currentDist += self.graphMatrix[permutation[i]][permutation[i + 1]]
            if (currentDist < minDist):
                minDist = currentDist
                minPath = permutation

        # Generate result from permutation
        for i in minPath:
            result.append(self.inputList[i]) 

        return result



    def permutations(self, elems):
        if len(elems) <= 1:
            yield elems
        else:
            for permutation in self.permutations(elems[1:]):
                for i in range(len(elems)):
                    yield permutation[:i] + elems[0:1] + permutation[i:]





class NewRailwayConstructorChristofides:

    '''
    I decided to implement the Christofides Algorithm because it allows to have a good approximation of the shortest new railway line by having an upper bound of approximation of 1.5*l, where l is the optimal length. This is achieved with time complexity O(N^2 logN), where N is the number of stations in the input, rather than O(N!) for TSP.
    '''

    def __init__(self, stations, inputList):
        # Construct a weighted graph in matrix format
        self.inputList = inputList
        self.stations = stations

        self.inputVertices = [i for i in range(len(this.inputList))]

        self.graph = self.buildCompleteGraph(len(self.inputVertices))

        self.msTree = self.minSpanningTree(self.graph)

        self.oddVertices = self.findOddDegreeVertices(self.msTree)





    def buildCompleteGraph(self, verticesNum):
        graph = WeightedGraph(len())
        for v in range verticesNum:
            for w in range verticesNum:
                if v == w: continue
                distance = euclideanDistance(v, w)
                edge = 
                graph.addDirectedEdge()

    def minSpanningTree(self, graph):
        primMST = PrimMST(graph)
        mst = primMST.edges()

    def findOddDegreeVertices(self, mst):
        pass

    def euclideanDistance(self, vertex1, vertex2):
        station1 = self.inputList[vertex1]
        station2 = self.inputList[vertex2]
        station1Value = self.stations.get(station1)
        station2Value = self.stations.get(station2)
        lat1 = station1Value[1]
        lon1 = station1Value[2]
        lat2 = station2Value[1]
        lon2 = station2Value[2]

        dist = math.sqrt( (lat1 - lat2)**2 + (lon1 - lon2)**2 )
        return dist






In [17]:
import csv

class LondonRailwayMapper(AbstractLondonRailwayMapper):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.graph = None
        self.stations = None
    
        
    def loadStationsAndLines(self):
        # ADD YOUR CODE HERE
        loader = DataLoader()
        self.graph, self.stations = loader.loadFiles()
    
    

    def minStops(self, fromS, toS):     
        numStops = -1
        # ADD YOUR CODE HERE
        finder = MinStopsBFS(self.graph)
        numStops = finder.minStops(self.stations.get(fromS)[0], self.stations.get(toS)[0])
        
        return numStops    
    
    
    
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        # ADD YOUR CODE HERE
        # In order to implement this requirement I decided to use the Dijkstra (binary heap) 
        # algorithm. This because it is O(E * logV) in typical 
        finder = ShortestPath(self.graph, self.stations.get(fromS)[0])
        minDistance = finder.findMinDistance(self.stations.get(toS)[0])
        
        return minDistance
    
    
    def newRailwayLine(self, inputList):
        outputList = []
        # ADD YOUR CODE HERE
        if (len(inputList) <= 20):
            newrailwayConstructor = NewRailwayConstructorTSP(self.stations, inputList)
            outputList = newrailwayConstructor.findNewLine()
        else:
            newrailwayConstructor = NewRailwayConstructorChristofides(self.stations, inputList)
            outputList = [] # TODO To be done
        
        return outputList

Use the cell below for all python code needed to test the `LondonRailwayMapper` class above.

In [18]:
import timeit

# ADD YOUR TEST CODE HERE

mapper = LondonRailwayMapper()
mapper.loadStationsAndLines()
# dict = mapper.stations

stops0 = mapper.minStops("Green Park", "Covent Garden")
stops1 = mapper.minStops("Covent Garden", "Green Park")
stops2 = mapper.minStops("Willesden Junction", "Upminster")
stops3 = mapper.minStops("Upminster", "Willesden Junction")
stops4 = mapper.minStops("Upminster", "Centrale")
stops5 = mapper.minStops("Centrale", "Upminster")
stops6 = mapper.minStops("Upminster", "Upminster")
stops7 = mapper.minStops("Regents Park", "Wembley Central")
stops8 = mapper.minStops("Wembley Central", "Regents Park")
stops9 = mapper.minStops("Barkingside", "Walthamstow Central")
stops10 = mapper.minStops("Walthamstow Central", "Barkingside")
stops11 = mapper.minStops("Harlesden", "Edgware Road (Bakerloo)")
stops12 = mapper.minStops("Edgware Road (Bakerloo)", "Harlesden")

dist0 = mapper.minDistance("Green Park", "Covent Garden")
dist1 = mapper.minDistance("Covent Garden", "Green Park")
dist2 = mapper.minDistance("Willesden Junction", "Upminster")
dist3 = mapper.minDistance("Upminster", "Willesden Junction")
dist4 = mapper.minDistance("Upminster", "Centrale")
dist5 = mapper.minDistance("Centrale", "Upminster")
dist6 = mapper.minDistance("Upminster", "Upminster")
dist7 = mapper.minDistance("Regents Park", "Wembley Central")
dist8 = mapper.minDistance("Wembley Central", "Regents Park")
dist9 = mapper.minDistance("Barkingside", "Walthamstow Central")
dist10 = mapper.minDistance("Walthamstow Central", "Barkingside")
dist11 = mapper.minDistance("Harlesden", "Edgware Road (Bakerloo)")
dist12 = mapper.minDistance("Edgware Road (Bakerloo)", "Harlesden")

print(stops0, stops1, stops2, stops3, stops4, stops5, stops6, stops7, stops8, stops9, stops10, stops11, stops12)
print(dist0, dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11, dist12)



# list0 = mapper.newRailwayLine(["Oxford Circus", "Barkingside", "Camden Town", "Shoreditch High Street"])
# list1 = mapper.newRailwayLine(["Oxford Circus", "Barkingside", "Shoreditch High Street", "Camden Town"])
# print(list0) 
# print(list1)
# print(list0 == list1)


# list0 = mapper.newRailwayLine(["Upminster", "Willesden Junction", "Camden Town", "Aldgate", "Chancery Lane", "Charing Cross", "Hither Green"])
# list1 = mapper.newRailwayLine(["Willesden Junction", "Upminster", "Camden Town", "Aldgate", "Chancery Lane", "Charing Cross", "Hither Green"])
# print(list0) 
# print(list1)
# print(list0 == list1)

# list0 = mapper.newRailwayLine(["Homerton", "Green Park", "Covent Garden", "Kilburn High Road", "London City Airport", "Mudchute", "Northumberland Park", "Preston Road", "Upminster"])
# list1 = mapper.newRailwayLine(["Kilburn High Road", "Homerton", "Green Park", "Covent Garden", "London City Airport", "Mudchute", "Northumberland Park", "Preston Road", "Upminster"])
# print(list0) 
# print(list1)
# print(list0 == list1)


# print("Stops:")

# print(stops0, stops1, stops2, stops3, stops4)
# print(dist0, dist1, dist2, dist3, dist4, dist5)

# starttime = timeit.default_timer()
# for i in range(0, 10000):
#     dict.get("Covent Garden")
# endtime = timeit.default_timer()
# print(endtime - starttime)


3 3 9 9 9 9 0 4 4 11 11 7 7
0.8778137608757194 0.8778137608757193 22.290930284989063 22.29093028498906 24.036207302755187 24.036207302755194 0 7.930052082202149 7.930052082202149 12.216696766720894 12.216696766720892 4.885447299215376 4.885447299215376


The cell below exemplifies the test code I will invoke on your submission. Do NOT modify it. 

In [19]:
# DO NOT MODIFY THIS CELL

import timeit

testMapper = LondonRailwayMapper()

#
# testing the loadStationsAndLines() API 
#
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

#
# testing the minStops() and minStops() API on a sample of from/to station pairs  
#
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")  
    
#
# testing the newRailwayLine() API on a small list of stations  
#
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

#
# testing the newRailwayLine() API on a big list of stations  
#
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


Execution time to load: 0.009

Execution time minStops: 0.002
Execution time minDistance: 0.002
From Baker Street to North Wembley in 6 stops and 6 miles

Execution time minStops: 0.001
Execution time minDistance: 0.002
From Epping to Belsize Park in 17 stops and 17 miles

Execution time minStops: 0.001
Execution time minDistance: 0.001
From Canonbury to Balham in 10 stops and 10 miles

Execution time minStops: 0.002
Execution time minDistance: 0.002
From Vauxhall to Leytonstone in 6 stops and 6 miles


Station list ['Queens Park', 'Chigwell', 'Moorgate', 'Swiss Cottage', 'Liverpool Street', 'Highgate']
New station line ['Chigwell', 'Liverpool Street', 'Moorgate', 'Highgate', 'Swiss Cottage', 'Queens Park']
Total track length from Chigwell to Queens Park : 18.710011238335046 miles
Execution time newLine: 0.001


NameError: name 'this' is not defined